# Decoding the Comments: Sentiment, Topics, and Predictive Insights

The comments section is often referred to as the "pulse" of online communities, offering an unfiltered view of audience sentiment, engagement, and reactions. In this final notebook of the project, we shift our focus to the comments dataset to uncover deeper insights about the audience and their interactions with video content.

Through a combination of **data analysis** and **machine learning techniques**, this notebook will explore the following key areas:
- **Sentiment Analysis**: Understanding the emotional tone of user comments and its correlation with video engagement.
- **Topic Modeling**: Uncovering recurring themes and patterns in audience discussions.
- **Predictive Modeling**: Using comments to predict engagement metrics and identify key drivers of interaction.
- **Advanced NLP**: Applying modern Natural Language Processing (NLP) techniques to classify sentiment, detect emotions, and extract nuanced insights.

This notebook not only aims to generate actionable insights but also demonstrates the application of machine learning and NLP tools to real-world data. By the end, we hope to bridge the gap between content creators and their audience, providing a clearer understanding of how comments reflect and influence engagement.

As the final notebook in this series, this analysis will tie together the insights from previous notebooks, offering a holistic view of how video content and audience interaction shape overall engagement trends.

In [1]:
!pip install isodate nltk wordcloud vaderSentiment scikit-learn pandas numpy matplotlib seaborn gensim xgboost spacy transformers textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 644.4 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:010m


In [3]:
# Data Manipulation
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser

# Text Processing and NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Topic Modeling and Advanced NLP (Optional)
#from gensim import corpora
#from gensim.models import LdaModel
#import spacy
#from transformers import pipeline
#import pyLDAvis.gensim_models as gensimvis

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid", color_codes=True)

# Utilities
from collections import Counter
from tqdm import tqdm

# Download NLTK data
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bashaar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/bashaar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
video_df = pd.read_csv("dataFolder/processed/cleanedDataFrame.csv")

In [7]:
comments_df = pd.read_csv("dataFolder/processed/cleanedComments.csv")